In [1]:
import numpy as np
import matplotlib.pyplot as plt
from seaborn import heatmap
import networkx as nx
import queue
import math

In [2]:
class Agent:
    pos = [0.0,0.0]
    direction = 0.0
    size = 3
    
    def __init__(self,x,y,direction):
        self.pos = [x,y]
        self.direction = direction
        
    def getSize(self):
        return self.size
    
    def getPos(self):
        return self.pos
    
    def getAproxPos(self):
        AproxPos = self.pos.copy()
        AproxPos[0] = round(AproxPos[0])
        AproxPos[1] = round(AproxPos[1])
        
        return AproxPos
    
    def getDirection(self):
        return self.direction
    
    def getCarFront(self):
        #E=0 or -PI, N=PI/2, W=PI, S= -(PI/2)
        front = self.getAproxPos()
        
        if self.direction > -(PI/8) and self.direction < PI/8:
            front[0] += 1
        elif self.direction > PI/8 and self.direction < 3*PI/8:
            front[0] += 1
            front[1] += 1
        elif self.direction > 3*PI/8 and self.direction < 5*PI/8:
            front[1] += 1
        elif self.direction > 5*PI/8 and self.direction < 7*PI/8:
            front[0] -= 1
            front[1] += 1
        elif self.direction > 7*PI/8 and self.direction < -(7*PI/8):
            front[0] -= 1
        elif self.direction > -(7*PI/8) and self.direction < -(5*PI/8):
            front[0] -= 1
            front[1] -= 1
        elif self.direction > -(5*PI/8) and self.direction < -(3*PI/8):
            front[1] -= 1
        elif self.direction > -(3*PI/8) and self.direction < -(1*PI/8):
            front[0] += 1
            front[1] -= 1
        else:
            print("outside PI direction values")
        
        return front
    
    def moveForward(self, step):
        #E=0 or -PI, N=PI/2, W=PI, S= -(PI/2)
        addx = math.cos(self.direction) * step
        addy = math.sin(self.direction) * step
        
        self.pos[0] = self.pos[0] + addx
        self.pos[1] = self.pos[1] + addy
            
    def turnOnSpot(self, direction):
        #-PI < direction < PI
        self.direction = self.direction + direction
        
        #Maintain between -PI < direction < PI
        if self.direction <= -PI:
            self.direction = self.direction + 2*PI
        if self.direction > PI:
            self.direction = self.direction - 2*PI
            
    def turnWhileMoving(self, step, radius):
        #E=0 or -PI, N=PI/2, W=PI, S= -(PI/2)
        self.pos[0] = self.pos[0] + step * math.cos(self.direction)
        self.pos[1] = self.pos[1] + step * math.sin(self.direction)
        self.direction = self.direction + (step/radius)
        
        if self.direction <= -PI:
            self.direction = self.direction + 2*PI
        if self.direction > PI:
            self.direction = self.direction - 2*PI

In [3]:
class Obstacle:
    pos = [0,0]
    face = 0
    #DEFINE Size of Obstacle
    size = 1
    
    def __init__(self, *args):
        if len(args) == 2:
            self.pos = [args[0], args[1]]
        elif len(args) == 3:
            self.pos = [args[0], args[1]]
            self.face = args[2]
        else:
            print("Incorrect argument count")
                
                
    def __eq__(self, other):
        if self.pos == other.pos and self.face == other.face:
            return True
        return False
    
    def getObsFacing(self):
        return self.face

In [4]:
class Map:
    mapGrid = []
    car = Agent(0,0,0)
    obsList = []
    obsDirDict = {}
    borders = set()
    size = 0
    
    global PI
    PI = math.pi
    
    #Step is the distance the robot can move in a straight line in one step
    steps = 1
    #radius is the distance from the circumference to the center circle when robot moves while turning
    radius = 2.5
    
    def __init__(self,size,x,y,direction):
        self.size = size + 2
        for i in range(self.size):
            self.borders.add((0,i))
            self.borders.add((self.size-1, i))
            self.borders.add((i,0))
            self.borders.add((i,self.size-1))
        #print(self.borders)
        self.initMap(self.size,x,y,direction)
    
    def initMap(self,size,x,y,direction):
        self.car.pos = [x,y]
        self.car.direction = direction
        for i in range(self.size):
            templist= []
            for j in range(self.size):
                if ((i,j) in self.borders):
                    templist.append("W")
                else:
                    templist.append("-")
            self.mapGrid.append(templist)
        self.markCar()
    
    def printMap(self):
        for i in range(self.size):
            for j in range(self.size):
                print(self.mapGrid[j][i], end = " ")
            print()
            
    def resetCar(self):
        loc = self.car.getAproxPos()
        sizeCar = self.car.getSize()
        #wingspan of the car
        span = int((sizeCar - 1)/2)
        
        #Remove car on map
        for i in range(-span, span+1):
            for j in range(-span, span+1):
                    self.mapGrid[loc[0]+i][loc[1]+j] = '-'
             
    
    def markCar(self):
        loc = self.car.getAproxPos()
        front = self.car.getCarFront()
        sizeCar = self.car.getSize()
        #wingspan of the car
        span = int((sizeCar - 1)/2)
        
        #Avoid checking more if one fails
        fail = 0
        #Mark car on map
        for i in range(-span, span+1):
            for j in range(-span, span+1):
                #Check whether car is in border or in Obstacle
                if (loc[0]+i>=0 and loc[0]+i<= self.size and loc[1]+j>=0 
                    and loc[1]+j<= self.size and (loc[0]+i,loc[1]+j) not in self.borders 
                    and Obstacle(loc[0]+i,loc[1]+j) not in self.obsList):
                     
                    if i == 0 and j == 0:
                        self.mapGrid[loc[0]][loc[1]] = '$'
                    elif (loc[0]+i) == front[0] and (loc[1]+j) == front[1]:
                        self.mapGrid[loc[0]+i][loc[1]+j] = '@'
                    else:
                        self.mapGrid[loc[0]+i][loc[1]+j] = '+'
                #Remove car if in border
                else:
                    fail = 1
                    break
            
            if fail == 1:
                    self.checkCarBorder()
                    print("Car in border obstacle")
                    break
    
    
    def checkCarBorder(self):
        loc = self.car.getAproxPos()
        sizeCar = self.car.getSize()
        #wingspan of the car
        span = int((sizeCar - 1)/2)
        
        #Remove car on map
        for i in range(-span, span+1):
            for j in range(-span, span+1):
                #Check whether car is in border
                if (loc[0]+i>=0 and loc[0]+i<=self.size and loc[1]+j>=0 
                    and loc[1]+j<=self.size):
                    if (loc[0]+i,loc[1]+j) in self.borders:
                        self.mapGrid[loc[0]+i][loc[1]+j] = 'W'
                    elif Obstacle(loc[0]+i,loc[1]+j) in self.obsList:
                        self.mapGrid[loc[0]+i][loc[1]+j] = self.obsDirDict[(loc[0]+i,loc[1]+j)]
                    else:
                        self.mapGrid[loc[0]+i][loc[1]+j] = '-'


    def moveForwardSteps(self, steps):
        self.resetCar()
        self.car.moveForward(steps)
        self.markCar() 
    
    def moveForwardOnce(self):
        self.resetCar()
        self.car.moveForward(self.steps)
        self.markCar()
        
    def turnCarOnSpot(self, rotate):
        self.resetCar()
        self.car.turnOnSpot(rotate)
        self.markCar()
        
    def turnCar(self, steps):
        self.resetCar()
        self.car.turnWhileMoving(steps, self.radius)
        self.markCar()
        
    
    def addObstacle(self,x,y,facing):
        if (x,y) not in self.borders:
            self.obsList.append(Obstacle(x,y))
            self.obsDirDict[(x,y)] = facing
            #print(self.obsDirDict)
            self.mapGrid[x][y] = facing
        else:
            print("Unable to add obstacle due to borders")

# Consider using dictionary to collate key-value pair for coordinates and direction?

In [5]:
maze = Map(20,2.0,19.0,0)

In [6]:
maze.printMap()

W W W W W W W W W W W W W W W W W W W W W W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W + + + - - - - - - - - - - - - - - - - - W 
W + $ @ - - - - - - - - - - - - - - - - - W 
W + + + - - - - - - - - - - - - - - - - - W 
W W W W W W W W W W W W W W W W W W W W W W 


In [7]:
maze.moveForwardOnce()

In [8]:
maze.printMap()

W W W W W W W W W W W W W W W W W W W W W W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - + + + - - - - - - - - - - - - - - - - W 
W - + $ @ - - - - - - - - - - - - - - - - W 
W - + + + - - - - - - - - - - - - - - - - W 
W W W W W W W W W W W W W W W W W W W W W W 


In [9]:
maze.turnCarOnSpot(-PI/4)
print(maze.car.direction)
maze.moveForwardSteps(10)
maze.printMap()

-0.7853981633974483
W W W W W W W W W W W W W W W W W W W W W W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - + + @ - - - - - - - - - W 
W - - - - - - - - + $ + - - - - - - - - - W 
W - - - - - - - - + + + - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W W W W W W W W W W W W W W W W W W

In [10]:
maze.turnCar(2)
maze.printMap()

W W W W W W W W W W W W W W W W W W W W W W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - + + + - - - - - - - - W 
W - - - - - - - - - + $ @ - - - - - - - - W 
W - - - - - - - - - + + + - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W W W W W W W W W W W W W W W W W W W W W W 


In [11]:
print(maze.car.direction)
maze.car.pos


0.014601836602551765


[11.485281374238571, 10.514718625761429]

In [12]:
maze.turnCar(5)
maze.printMap()

W W W W W W W W W W W W W W W W W W W W W W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - + + + - - - W 
W - - - - - - - - - - - - - - + $ + - - - W 
W - - - - - - - - - - - - - - @ + + - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W W W W W W W W W W W W W W W W W W W W W W 


In [13]:
print(maze.car.direction)
maze.car.pos

2.0146018366025515


[16.48474834962893, 10.58772521437633]

In [14]:
maze.moveForwardOnce()
maze.moveForwardOnce()
maze.moveForwardOnce()
print(maze.car.direction)
maze.car.pos

2.0146018366025515


[15.196610107989255, 13.297097811570273]

In [15]:
maze.printMap()

W W W W W W W W W W W W W W W W W W W W W W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - + + + - - - - W 
W - - - - - - - - - - - - - + $ + - - - - W 
W - - - - - - - - - - - - - @ + + - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W W W W W W W W W W W W W W W W W W W W W W 


In [16]:
maze.addObstacle(2,2,"V")

In [17]:
maze.addObstacle(5,5,"^")

In [18]:
print(Obstacle(2,2) in maze.obsList)

True


In [19]:
maze.addObstacle(0,0,0)

Unable to add obstacle due to borders


In [20]:
maze.printMap()

W W W W W W W W W W W W W W W W W W W W W W 
W - - - - - - - - - - - - - - - - - - - - W 
W - V - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - ^ - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - + + + - - - - W 
W - - - - - - - - - - - - - + $ + - - - - W 
W - - - - - - - - - - - - - @ + + - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W W W W W W W W W W W W W W W W W W W W W W 


### Will need to account for size of wall